In [18]:
#dependencies
import pandas as pd
import numpy as np
import requests
from citipy import citipy
from config import weather_api_key


In [19]:
#create 2000 random lats and lngs using np.random.uniform function
lats = np.random.uniform(low=-90, high=90, size = 2000)
lngs = np.random.uniform(low=-180, high=180, size = 2000)
lat_lng_zip = zip(lats,lngs)


In [20]:
#put the zip paired lats and lngs in a list and unzip
coordinates = list(lat_lng_zip)

In [21]:
#create a list holding cities
cities = []
#find cities near the coordinates using citipy
for c in coordinates:
    city = citipy.nearest_city(c[0],c[1]).city_name
    #if the city is not already in the cities list, add it in
    if city not in cities:
        cities.append(city)
len(cities)

741

In [26]:
#try out one city to see the json file
#make API call from openweathermap site
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Metric&q=beijing&appid=" + weather_api_key

beijing_eg = requests.get(base_url).json()
beijing_eg

{'coord': {'lon': 116.3972, 'lat': 39.9075},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 8.64,
  'feels_like': 4.6,
  'temp_min': 6.67,
  'temp_max': 11.11,
  'pressure': 1013,
  'humidity': 18},
 'visibility': 10000,
 'wind': {'speed': 1, 'deg': 0},
 'clouds': {'all': 0},
 'dt': 1612496690,
 'sys': {'type': 1,
  'id': 9609,
  'country': 'CN',
  'sunrise': 1612480747,
  'sunset': 1612517861},
 'timezone': 28800,
 'id': 1816670,
 'name': 'Beijing',
 'cod': 200}

In [30]:
#make API call from openweathermap site
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Metric&appid=" + weather_api_key


#create an empty list to hold the weather data
city_data = []
#print the beginning of the logging
print("Beginning Data Retrieval")
print("---------------------------")

#create counters
record_count = 1
set_count = 1

#loop thru all the cities in the cities list
for i, city in enumerate(cities):

    #group cities in sets of 50 for logging purposes
    #if the index can be devided by 50 and index number is greater than 50, then add set_count with 1
    if(i % 50 == 0 and i>50):
        set_count += 1
        record_count = 1
    #create endpoint url with each city, if city name has space in between, replace with + sign
    city_url = base_url + "&q=" + city.replace(" ","+")

    #log the url, record, and set numbers and the city 
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    #add 1 to the record count
    record_count += 1

    #run an API request for each cities with try except for errors
    try:
        city_weather = requests.get(city_url).json()
        #parce the JSON and retrieved data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_desp = city_weather["weather"][0]["description"]

        #append the city information into City_data list
        city_data.append({
            "City": city.title(),
            "Country": city_country,
            "Lat": city_lat,
            "Lng": city_lng,
            "Max Temp": city_max_temp,
            "Humidity": city_humidity,
            "Cloudiness": city_clouds,
            "Wind Speed": city_wind,
            "Current Description": city_weather_desp})
            
    #if an error occurs, skip the city
    except:
        print("City not found, skipping....")
        pass
    #indicate that Data Loading is complete
    print("------------------")
    print("Data Retrieval Complete")
    print("------------------")

indoni
------------------
Data Retrieval Complete
------------------
Processing Record 3 of Set 11 | ngunguru
------------------
Data Retrieval Complete
------------------
Processing Record 4 of Set 11 | svetlyy
------------------
Data Retrieval Complete
------------------
Processing Record 5 of Set 11 | roma
------------------
Data Retrieval Complete
------------------
Processing Record 6 of Set 11 | labuan
------------------
Data Retrieval Complete
------------------
Processing Record 7 of Set 11 | lexington
------------------
Data Retrieval Complete
------------------
Processing Record 8 of Set 11 | ambunti
------------------
Data Retrieval Complete
------------------
Processing Record 9 of Set 11 | mirabad
------------------
Data Retrieval Complete
------------------
Processing Record 10 of Set 11 | seydi
------------------
Data Retrieval Complete
------------------
Processing Record 11 of Set 11 | waddan
------------------
Data Retrieval Complete
------------------
Processing Reco

In [31]:
#check how many cities has data response from the API call
len(city_data)

688

In [32]:
#create datafrom using pandas
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hithadhoo,MV,-0.6000,73.0833,27.96,78,100,5.35,overcast clouds
1,Punta Arenas,CL,-53.1500,-70.9167,9.00,61,75,5.14,broken clouds
2,Dharchula,IN,29.8500,80.5333,11.98,47,97,1.34,overcast clouds
3,Albany,US,42.6001,-73.9662,-3.33,64,27,1.19,scattered clouds
4,Kloulklubed,PW,7.0419,134.2556,30.00,70,75,4.63,broken clouds


In [33]:
#Create the output file csv
output_csv = city_data_df.to_csv("WeatherPy_Database.csv",index=False)